# Block Aligner Benchmark Analysis and Visualizations

This notebook contains code for collecting, cleaning, and analyzing data produced by block aligner's experiments.

To run this, you will need to install all the libraries imported below, along with [altair-saver](https://github.com/altair-viz/altair_saver) and [altair-data-server](https://github.com/altair-viz/altair_data_server), which has some extra dependencies for PDF saving.

Run each cell one by one to reproduce the experiments. This may take a while. For accurate benchmarking, it is recommended to run the entire notebook in the command line with `nbconvert`.

In [1]:
import altair as alt
from altair_saver import save
from altair import datum
import pandas as pd
from io import StringIO

alt.data_transformers.enable("data_server")

DataTransformerRegistry.enable('data_server')

In [2]:
def csv_to_pandas(csv, d = "\\s*,\\s*", t = None):
    s = StringIO("\n".join(csv))
    data = pd.read_csv(s, sep = d, thousands = t, comment = "#", engine = "python")
    return data

## Prefix Scan Benchmark

In [3]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- prefix_scan | grep 'bench:' | awk '{print $2"\t"$5}'
output.insert(0, "algorithm\ttime")
output

['algorithm\ttime', 'bench_naive_prefix_scan\t34', 'bench_opt_prefix_scan\t17']

In [4]:
data = csv_to_pandas(output, d = "\t", t = ",")
data

,algorithm,time
0,bench_naive_prefix_scan,34
1,bench_opt_prefix_scan,17


In [5]:
data["algorithm"] = data["algorithm"].map({
    "bench_naive_prefix_scan": "naive",
    "bench_opt_prefix_scan": "ours"
})
data

,algorithm,time
0,naive,34
1,ours,17


Prefix Scan Benchmark (AVX2)

In [6]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("time", axis = alt.Axis(title = "time (ns)")),
    y = "algorithm",
    color = alt.Color("algorithm", legend = None)
).properties(
    width = 150
)
save(c, "prefix_scan_bench.pdf")
c

alt.Chart(...)

## Random Data Benchmark

In [7]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- bench_ | grep 'bench:' | grep -v 'prefix_scan' | awk '{print $2"\t"$5}'
output

['bench_parasailors_aa_1000_10000\t47,752,086',
 'bench_parasailors_aa_100_1000\t500,261',
 'bench_parasailors_aa_10_100\t17,732',
 'bench_rustbio_aa_100_1000\t14,464,156',
 'bench_rustbio_aa_10_100\t159,708',
 'bench_scan_aa_1000_10000\t238,002',
 'bench_scan_aa_1000_10000_insert\t8,877,013',
 'bench_scan_aa_1000_10000_small\t221,131',
 'bench_scan_aa_1000_10000_trace\t1,923,387',
 'bench_scan_aa_100_1000\t24,781',
 'bench_scan_aa_100_1000_insert\t51,456',
 'bench_scan_aa_100_1000_small\t23,602',
 'bench_scan_aa_100_1000_trace\t515,355',
 'bench_scan_aa_10_100\t3,962',
 'bench_scan_aa_10_100_insert\t4,121',
 'bench_scan_aa_10_100_small\t3,468',
 'bench_scan_aa_10_100_trace\t361,087',
 'bench_scan_nuc_1000_10000\t222,506',
 'bench_scan_nuc_100_1000\t22,688',
 'bench_triple_accel_1000_10000\t8,325,161',
 'bench_triple_accel_100_1000\t25,337']

In [8]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["parasailors_aa", "rustbio_aa", "scan_aa", "scan_nuc", "triple_accel"]
new_names = ["parasailors\tprotein", "rust bio\tprotein", "ours\tprotein", "ours\tnucleotide", "triple accel\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'parasailors\tprotein\t1000\t10000\tdefault\t47,752,086',
 'parasailors\tprotein\t100\t1000\tdefault\t500,261',
 'parasailors\tprotein\t10\t100\tdefault\t17,732',
 'rust bio\tprotein\t100\t1000\tdefault\t14,464,156',
 'rust bio\tprotein\t10\t100\tdefault\t159,708',
 'ours\tprotein\t1000\t10000\tdefault\t238,002',
 'ours\tprotein\t1000\t10000\tinsert\t8,877,013',
 'ours\tprotein\t1000\t10000\tsmall\t221,131',
 'ours\tprotein\t1000\t10000\ttrace\t1,923,387',
 'ours\tprotein\t100\t1000\tdefault\t24,781',
 'ours\tprotein\t100\t1000\tinsert\t51,456',
 'ours\tprotein\t100\t1000\tsmall\t23,602',
 'ours\tprotein\t100\t1000\ttrace\t515,355',
 'ours\tprotein\t10\t100\tdefault\t3,962',
 'ours\tprotein\t10\t100\tinsert\t4,121',
 'ours\tprotein\t10\t100\tsmall\t3,468',
 'ours\tprotein\t10\t100\ttrace\t361,087',
 'ours\tnucleotide\t1000\t10000\tdefault\t222,506',
 'ours\tnucleotide\t100\t1000\tdefault\t22,688',
 'triple accel\tnucleotide\t1000\t100

In [9]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,parasailors,protein,1000,10000,default,47752086
1,parasailors,protein,100,1000,default,500261
2,parasailors,protein,10,100,default,17732
3,rust bio,protein,100,1000,default,14464156
4,rust bio,protein,10,100,default,159708
5,ours,protein,1000,10000,default,238002
6,ours,protein,1000,10000,insert,8877013
7,ours,protein,1000,10000,small,221131
8,ours,protein,1000,10000,trace,1923387
9,ours,protein,100,1000,default,24781


In [10]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (AVX2)

In [11]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench.pdf")
c

alt.Chart(...)

Random DNA Sequences Benchmark (AVX2)

In [12]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000])),
    shape = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000]))
).transform_filter(
    datum.alphabet == "nucleotide"
).properties(
    width = 200,
    height = 50
)
save(c, "random_dna_bench.pdf")
c

alt.Chart(...)

## Uniclust 30 Data Benchmark

In [13]:
output = !cd .. && cargo run --example uc_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, size, time',
 'ours (no trace), uc30, 32-32, 0.056015377',
 'ours (no trace), uc30 0.95, 32-32, 0.059886296',
 'ours (no trace), uc30, 32-256, 0.090234463',
 'ours (no trace), uc30 0.95, 32-256, 0.076965265',
 'ours (no trace), uc30, 256-256, 0.199199573',
 'ours (no trace), uc30 0.95, 256-256, 0.218014174',
 'ours (trace), uc30, 32-256, 0.1829081',
 'ours (trace), uc30 0.95, 32-256, 0.162208597',
 'parasail, uc30, full, 0.837155858',
 'parasail, uc30 0.95, full, 0.974040094']

In [14]:
data = csv_to_pandas(output)
data

,algorithm,dataset,size,time
0,ours (no trace),uc30,32-32,0.056015
1,ours (no trace),uc30 0.95,32-32,0.059886
2,ours (no trace),uc30,32-256,0.090234
3,ours (no trace),uc30 0.95,32-256,0.076965
4,ours (no trace),uc30,256-256,0.199200
5,ours (no trace),uc30 0.95,256-256,0.218014
6,ours (trace),uc30,32-256,0.182908
7,ours (trace),uc30 0.95,32-256,0.162209
8,parasail,uc30,full,0.837156
9,parasail,uc30 0.95,full,0.974040


Uniclust30 Benchmark (AVX2)

In [15]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", axis = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = "algorithm"
).transform_filter(
    (datum.size == "32-256") | (datum.algorithm == "parasail")
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
).configure_range(
    category = {"scheme": "dark2"}
)
save(c, "uniclust30_bench.pdf")
c

alt.FacetChart(...)

Uniclust30 Block Size Benchmark (AVX2)

In [16]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", axis = None, sort = ["32-32", "32-256", "256-256"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = alt.Color("size", sort = ["32-32", "32-256", "256-256"])
).transform_filter(
    datum.algorithm == "ours (no trace)"
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
)
save(c, "uniclust30_size_bench.pdf")
c

alt.FacetChart(...)

## Nanopore Data Benchmark Setup

To run the benchmarks below, you need to clone the following repos, place them in the same directory where this repo (block aligner) is located, and follow their setup instructions:
* [diff-bench-paper](https://github.com/Daniel-Liu-c0deb0t/diff-bench-paper)
* [adaptivebandbench](https://github.com/Daniel-Liu-c0deb0t/adaptivebandbench)

## Nanopore Data Benchmark

In [17]:
output = !cd .. && cargo run --example nanopore_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, time',
 'ours (no trace 32-32), nanopore 25kbp, 0.958155512',
 'ours (no trace 32-32), random, 2.382894137',
 'ours (trace 32-32), nanopore 25kbp, 1.350374066',
 'ours (trace 32-32), random, 3.291213083',
 'ours (trace 32-64), nanopore 25kbp, 1.634524491',
 'ours (trace 32-64), random, 3.500535939']

In [18]:
data = csv_to_pandas(output)
data

,algorithm,dataset,time
0,ours (no trace 32-32),nanopore 25kbp,0.958156
1,ours (no trace 32-32),random,2.382894
2,ours (trace 32-32),nanopore 25kbp,1.350374
3,ours (trace 32-32),random,3.291213
4,ours (trace 32-64),nanopore 25kbp,1.634524
5,ours (trace 32-64),random,3.500536


In [19]:
output2 = !cd ../../diff-bench-paper/supplementary_data/benchmark_codes && ./custom_bench.sh

for i, o in enumerate(output2):
    if o.startswith("cells("):
        break
output2 = output2[i + 1:]

output2.insert(0, "algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail")
output2

['algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail',
 'editdist\t466476000\t166711000\t66694000\t699881000\t6880489\t0',
 'non-diff\t672802000\t258715000\t60206000\t991723000\t27124786\t52',
 'diff-raw\t611988000\t204682000\t62590000\t879260000\t27291141\t32',
 'libgaba\t433386000\t149988000\t31496000\t614870000\t27121546\t53',
 'edlib\t28037720000\t19375110000\t106468000\t47519298000\t37\t0',
 'seqan\t90229880000\t0\t0\t90229880000\t0\t0']

In [20]:
data2 = csv_to_pandas(output2, d = "\t")
data2

,algorithm,fill time,trace time,convert time,total time,score,fail
0,editdist,466476000,166711000,66694000,699881000,6880489,0
1,non-diff,672802000,258715000,60206000,991723000,27124786,52
2,diff-raw,611988000,204682000,62590000,879260000,27291141,32
3,libgaba,433386000,149988000,31496000,614870000,27121546,53
4,edlib,28037720000,19375110000,106468000,47519298000,37,0
5,seqan,90229880000,0,0,90229880000,0,0


In [21]:
cleaned2 = data2.drop(columns = ["trace time", "convert time", "total time", "score", "fail"])
cleaned2 = cleaned2.rename(columns = {"fill time": "time"})
cleaned2["time"] /= 1e9
cleaned2

,algorithm,time
0,editdist,0.466476
1,non-diff,0.672802
2,diff-raw,0.611988
3,libgaba,0.433386
4,edlib,28.037720
5,seqan,90.229880


In [22]:
cleaned = data.drop(index = [1, 3, 5])
cleaned = cleaned.drop(columns = ["dataset"])
cleaned = cleaned.append(cleaned2, ignore_index = True)
cleaned

,algorithm,time
0,ours (no trace 32-32),0.958156
1,ours (trace 32-32),1.350374
2,ours (trace 32-64),1.634524
3,editdist,0.466476
4,non-diff,0.672802
5,diff-raw,0.611988
6,libgaba,0.433386
7,edlib,28.037720
8,seqan,90.229880


25kbp Nanopore Reads Benchmark (AVX2)

In [23]:
chart1 = alt.Chart(cleaned).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm != "ours (trace 32-32)") & (datum.algorithm != "ours (no trace 32-32)") & (datum.algorithm != "ours (trace 32-64)"))

chart2 = alt.Chart(cleaned).mark_point(color = "red", filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm == "ours (trace 32-32)") | (datum.algorithm == "ours (no trace 32-32)") | (datum.algorithm == "ours (trace 32-64)"))

c = (chart1 + chart2).properties(
    width = 150,
    height = 150
)
save(c, "nanopore_bench.pdf")
c

alt.LayerChart(...)

## Sequence-to-Profile Alignment Benchmark

In [24]:
output = !cd .. && cargo run --example pssm_bench --release --features simd_avx2 --quiet
output

['size, time',
 '32-32, 0.159940944',
 '32-64, 0.185192795',
 '32-128, 0.217389978',
 '128-128, 0.270265633',
 'parasail, 0.620794405',
 '# Done!']

In [25]:
data = csv_to_pandas(output)
data

,size,time
0,32-32,0.159941
1,32-64,0.185193
2,32-128,0.217390
3,128-128,0.270266
4,parasail,0.620794


SCOP Sequence-to-Profile Alignment Benchmark (AVX2)

In [26]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("size", sort = ["32-32", "32-64", "32-128", "128-128", "parasail"], legend = None)
).transform_filter(
    datum.size != "2048-2048"
).properties(
    width = 75,
    height = 100
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = c + t
save(c, "pssm_size_bench.pdf")
c

alt.LayerChart(...)

## WASM SIMD

[Wasmtime](https://wasmtime.dev/) is needed to run the webassembly code.

In [27]:
output = !CARGO_TARGET_WASM32_WASI_RUNNER="wasmtime --wasm-features simd --" cargo bench --target=wasm32-wasi --features simd_wasm --quiet -- --nocapture | grep 'bench:' | awk '{print $2"\t"$5}'
output

['bench_rustbio_aa_100_1000\t23,525,319',
 'bench_rustbio_aa_10_100\t244,772',
 'bench_scan_aa_1000_10000\t998,361',
 'bench_scan_aa_1000_10000_insert\t11,536,279',
 'bench_scan_aa_1000_10000_small\t667,754',
 'bench_scan_aa_1000_10000_trace\t2,686,671',
 'bench_scan_aa_100_1000\t92,798',
 'bench_scan_aa_100_1000_insert\t173,512',
 'bench_scan_aa_100_1000_small\t64,112',
 'bench_scan_aa_100_1000_trace\t394,792',
 'bench_scan_aa_10_100\t8,286',
 'bench_scan_aa_10_100_insert\t11,915',
 'bench_scan_aa_10_100_small\t5,910',
 'bench_scan_aa_10_100_trace\t192,248',
 'bench_scan_nuc_1000_10000\t665,329',
 'bench_scan_nuc_100_1000\t67,545']

In [28]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["rustbio_aa", "scan_aa", "scan_nuc"]
new_names = ["rust bio\tprotein", "ours\tprotein", "ours\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'rust bio\tprotein\t100\t1000\tdefault\t23,525,319',
 'rust bio\tprotein\t10\t100\tdefault\t244,772',
 'ours\tprotein\t1000\t10000\tdefault\t998,361',
 'ours\tprotein\t1000\t10000\tinsert\t11,536,279',
 'ours\tprotein\t1000\t10000\tsmall\t667,754',
 'ours\tprotein\t1000\t10000\ttrace\t2,686,671',
 'ours\tprotein\t100\t1000\tdefault\t92,798',
 'ours\tprotein\t100\t1000\tinsert\t173,512',
 'ours\tprotein\t100\t1000\tsmall\t64,112',
 'ours\tprotein\t100\t1000\ttrace\t394,792',
 'ours\tprotein\t10\t100\tdefault\t8,286',
 'ours\tprotein\t10\t100\tinsert\t11,915',
 'ours\tprotein\t10\t100\tsmall\t5,910',
 'ours\tprotein\t10\t100\ttrace\t192,248',
 'ours\tnucleotide\t1000\t10000\tdefault\t665,329',
 'ours\tnucleotide\t100\t1000\tdefault\t67,545']

In [29]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,rust bio,protein,100,1000,default,23525319
1,rust bio,protein,10,100,default,244772
2,ours,protein,1000,10000,default,998361
3,ours,protein,1000,10000,insert,11536279
4,ours,protein,1000,10000,small,667754
5,ours,protein,1000,10000,trace,2686671
6,ours,protein,100,1000,default,92798
7,ours,protein,100,1000,insert,173512
8,ours,protein,100,1000,small,64112
9,ours,protein,100,1000,trace,394792


In [30]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (WASM SIMD)

In [31]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench_wasm.pdf")
c

alt.Chart(...)